In [5]:
import define_transformations_afolu as dta
import define_transformations_circular_economy as dtc
import define_transformations_energy as dte
import define_transformations_ippu as dti
import define_transformations_integrated as dtr

import os, os.path
import numpy as np
import pandas as pd
#from model_attributes import *
import setup_analysis as sa
import sisepuede_data_api as api
import sisepuede_models as sm
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
from typing import Union
import inspect
import ingestion as ing
import logging
from sisepuede_file_structure import *
import transformations_base_afolu as tba
import transformations_base_circular_economy as tbc
import transformations_base_energy as tbe
import transformations_base_general as tbg
import transformations_base_ippu as tbi



import sisepuede as ssp
warnings.filterwarnings("ignore")

def _setup_logger(namespace: str, fn_out: Union[str, None] = None) -> None:
    global logger
    
    format_str = "%(asctime)s - %(levelname)s - %(message)s"
    # configure
    if fn_out is not None:
        logging.basicConfig(
            filename = fn_out,
            filemode = "w",
            format = format_str,
            level = logging.DEBUG
        )
    else:
        logging.basicConfig(
            format = format_str,
            level = logging.DEBUG
        )
        
    logger = logging.getLogger(namespace)
    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter
    formatter = logging.Formatter(format_str)
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)

    return logger

_setup_logger(__name__, os.path.join(os.getcwd(), "log_convert_primaries_to_strategy.log"))


<Logger __main__ (DEBUG)>

In [3]:
regions_lac = [
    "argentina",
    "bahamas",
    "barbados",
    "belize",
    "bolivia",
    "brazil",
    "chile",
    "colombia",
    "costa_rica",
    "dominican_republic",
    "ecuador",
    "el_salvador",
    "guatemala",
    "guyana",
    "haiti",
    "honduras",
    "jamaica",
    "mexico",
    "nicaragua",
    "panama",
    "paraguay",
    "peru",
    "suriname",
    "trinidad_and_tobago",
    "uruguay",
    "venezuela"
]



year_0_ramp = 2025
dir_calibs = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/calibrated_input_files_from_edmundo"
fp_inputs = os.path.join(dir_calibs, "input_all_sectors.csv")
df_input = pd.read_csv(fp_inputs)
field_region = "nation"

regions_run = [
    x for x in sorted(regions_lac) if x in list(df_input[field_region].unique())
]

# reduce inputs
df_input = df_input[
    df_input[field_region].isin(regions_run)
].reset_index(drop = True)

warnings.filterwarnings("ignore")

In [8]:

import sisepuede as ssp
sisepuede = ssp.SISEPUEDE(
    "calibrated", 
    id_str = "sisepuede_run_2023-10-06T20:10:09.932282",
    logger = logger,
    regions = regions_run,
)



# STRATS TO RUN

attr_strat = sisepuede.model_attributes.get_dimensional_attribute_table(sisepuede.key_strategy)
dict_code_to_id = attr_strat.field_maps.get(f"strategy_code_to_{attr_strat.key}")
strat_codes_ignore = [
    # AFOLU
    "LNDU:INC_LAND_REHABILITIATION",
    "LSMM:INC_MANAGEMENT_CATTLE_PIGS",
    "LSMM:INC_MANAGEMENT_OTHER",
    "LSMM:INC_MANAGEMENT_POULTRY",
    "LSMM:INC_CAPTURE_BIOGAS",
    "TRWW:INC_COMPLIANCE_SEPTIC",
    "IPPU:DEC_OTHER_FCS",
    "IPPU:DEC_HFCS",
    "IPPU:DEC_PFCS",
    
    # ENERGY
    "TRNS:BUNDLE_DEMAND_MANAGEMENT",
    "TRNS:BUNDLE_DEMAND_MANAGEMENT_REP",
    "TRNS:BUNDLE_EFFICIENCY",
    "TRNS:BUNDLE_EFFICIENCY_REP",
    "TRNS:BUNDLE_FUEL_SWITCH",
    "TRNS:BUNDLE_FUEL_SWITCH_REP",
    "TRNS:BUNDLE_MODE_SHIFT",
    "TRNS:BUNDLE_MODE_SHIFT_REP"
]
strat_ids_drop = [
    dict_code_to_id.get(x) for x in strat_codes_ignore
]
strats_run = [x for x in attr_strat.key_values if x not in strat_ids_drop]




2023-10-07 22:01:29,087 - INFO - Successfully initialized SISEPUEDEFileStructure.
2023-10-07 22:01:29,091 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2023-10-07 22:01:29,091 - INFO - 	Setting export engine to 'sqlite'.
2023-10-07 22:01:29,114 - DEBUG - Trying to get columns for ANALYSIS_METADATA with query: select * from ANALYSIS_METADATA limit 0;
2023-10-07 22:01:29,118 - INFO - Table ANALYSIS_METADATA found in sql connection; it will not be replaced.
2023-10-07 22:01:29,119 - INFO - 	Columns successfully retrieved.
2023-10-07 22:01:29,119 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2023-10-07 22:01:29,120 - INFO - Successfully instantiated table ANALYSIS_METADATA
2023-10-07 22:01:29,122 - DEBUG - Trying to get columns for ATTRIBUTE_DESIGN with query: select * from ATTRIBUTE_DESIGN limit 0;
2023-10-07 22:01:29,125 - INFO - Table ATTRIB

# Run experiment (October 7 2023)
- response to reviewers
- Edmundo identified 5 primary ids from this set: `sisepuede_run_2023-10-06T20:10:09.932282`
- read these in, generate inputs, and map to templates


In [92]:
# read in data frame
fp_primaries_to_strategies = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/experiments_and_analysis/sisepuede_run_2023-10-06T20;10;09.932282/primary_ids_from_experiment_as_strategies;selection_for_uncertainty.csv"
df_primaries_to_strategies = pd.read_csv(fp_primaries_to_strategies)


# next, map primaries to new stratgy ids
primaries = sorted(list(df_primaries_to_strategies[sisepuede.key_primary]))
dict_primaries_to_strategies = dict((x, 5011 + i) for i, x in enumerate(primaries))
# strategy to which lhs-ls are applied
strategy_devation = 5009



def get_lhs_trajes(
    dict_filt: dict,
    region: str,
    sisepuede: ssp.SISEPUEDE,
    lhs_design: Union['LHSDesign', None] = None,
) -> Tuple[pd.Series, pd.Series, bool]:
    """
    Get LHS trajectories for input to generator
    """
    future_trajectories_cur = sisepuede.experimental_manager.dict_future_trajectories.get(region)
    lhs_design_cur = (
        sisepuede.experimental_manager.dict_lhs_design.get(region)
        if lhs_design is None
        else lhs_design
    )
    region_out = sisepuede.get_output_region(region)

    design = dict_filt.get(sisepuede.key_design)
    future = dict_filt.get(sisepuede.key_future)
    strategy = dict_filt.get(sisepuede.key_strategy)

    df_lhs_l, df_lhs_x = lhs_design_cur.retrieve_lhs_tables_by_design(design, return_type = pd.DataFrame)

    # reduce lhs tables - LEs
    df_lhs_l = (
        df_lhs_l[df_lhs_l[sisepuede.key_future].isin([future])] 
        if (df_lhs_l is not None) 
        else df_lhs_l
    )
    # Xs
    df_lhs_x = (
        df_lhs_x[df_lhs_x[sisepuede.key_future].isin([future])] 
        if (df_lhs_x is not None) 
        else df_lhs_x
    )


    ##  GENERATE INPUT BY FUTURE

    # determine if baseline future and fetch lhs rows
    base_future_q = (future == sisepuede.baseline_future)
    lhs_l = (
        df_lhs_l[df_lhs_l[sisepuede.key_future] == future].iloc[0] 
        if ((df_lhs_l is not None) and not base_future_q) 
        else None
    )
    lhs_x = (
        df_lhs_x[df_lhs_x[sisepuede.key_future] == future].iloc[0] 
        if ((df_lhs_x is not None) and not base_future_q) 
        else None
    )
    
    tup_out = lhs_l, lhs_x, base_future_q
    
    return tup_out



lhs_design = sisepuede.experimental_manager.dict_lhs_design.get(sorted(regions_run)[0])
df_out = []
t0 = time.time()
sep = "#"*40 + "\n"

for region in regions_run:
    
    ftc = sisepuede.experimental_manager.dict_future_trajectories.get(region)
    
    for primary_id in primaries:
        
        # get new strategy id and filter based on primary IDs selected to convert to strategies
        strat_new = dict_primaries_to_strategies.get(primary_id)
        dict_filt_cur = sisepuede.odpt_primary.get_dims_from_key(
            primary_id,
            return_type = "dict",
        )
        
        
        lhs_l, lhs_x, base_future_q = get_lhs_trajes(
            dict_filt_cur,
            region,
            sisepuede,
            lhs_design = lhs_design,
        )

        # build inputs
        df_in = ftc.generate_future_from_lhs_vector(
            lhs_x, 
            df_row_lhc_sample_l = lhs_l,
            future_id = None, 
            baseline_future_q = base_future_q,
        )
        
        # filter and update
        df_in = (
            df_in[
                df_in[sisepuede.key_strategy].isin([strategy_devation])
            ]
            .reset_index(drop = True)
        )
        df_in[sisepuede.key_strategy] = strat_new
        df_in[sisepuede.key_region] = region
        
        df_out.append(df_in)
        
        print(f"primary_id {primary_id} in region {region} done.")
     
    
    t_elapse = sf.get_time_elapsed(t0)
    print(f"\n{sep}#\n#\tRegion '{region}' complete at {t_elapse} seconds\n#\n{sep}\n\n")
    
        
df_out = (
    pd.concat(df_out, axis = 0)
    .sort_values(by = [sisepuede.key_region, sisepuede.key_strategy])
    .reset_index(drop = True)
)

fp_out = os.path.join(
    os.path.dirname(fp_primaries_to_strategies),
    "strategies_from_primaries_long.csv"
)

df_out.to_csv(
    fp_out,
    index = None,
    encoding = "UTF-8",
)


primary_id 514914 in region argentina done.
primary_id 514926 in region argentina done.
primary_id 514978 in region argentina done.
primary_id 515112 in region argentina done.
primary_id 515483 in region argentina done.

########################################
#
#	Region 'argentina' complete at 32.96 seconds
#
########################################



primary_id 514914 in region bolivia done.
primary_id 514926 in region bolivia done.
primary_id 514978 in region bolivia done.
primary_id 515112 in region bolivia done.
primary_id 515483 in region bolivia done.

########################################
#
#	Region 'bolivia' complete at 66.97 seconds
#
########################################



primary_id 514914 in region brazil done.
primary_id 514926 in region brazil done.
primary_id 514978 in region brazil done.
primary_id 515112 in region brazil done.
primary_id 515483 in region brazil done.

########################################
#
#	Region 'brazil' complete at 101.36 seconds
#
##

In [61]:
lhs_l.get(38)

lhs2 = lhs_l.to_dict().copy()
lhs2.update({38:0.4})
lhs2 = pd.Series(lhs2)

# build inputs
df_in = ftc.generate_future_from_lhs_vector(
    lhs_x, 
    df_row_lhc_sample_l = lhs2,
    future_id = None, 
    baseline_future_q = base_future_q,
)

df_in[df_in["strategy_id"].isin([5009])][[x for x in df_in.columns if x.startswith("pij_lndu_forests_primary_to")]].tail()

,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_primary_to_forests_mangroves,pij_lndu_forests_primary_to_forests_primary,pij_lndu_forests_primary_to_forests_secondary,pij_lndu_forests_primary_to_grasslands,pij_lndu_forests_primary_to_other,pij_lndu_forests_primary_to_settlements,pij_lndu_forests_primary_to_wetlands
4603,0.000021,0.0,0.991301,0.000589,0.004089,0.001389,0.002141,0.000471
4604,0.000021,0.0,0.991131,0.000603,0.004194,0.001389,0.002192,0.000471
4605,0.000021,0.0,0.990961,0.000617,0.004299,0.001388,0.002243,0.000471
4606,0.000022,0.0,0.990791,0.000631,0.004405,0.001387,0.002294,0.000471
4607,0.000022,0.0,0.990621,0.000645,0.004510,0.001386,0.002345,0.000471


In [76]:
#sisepuede.model_attributes.configuration.get("random_seed")

[1, 2] == [1, 2]

True

In [78]:
d_arg = sisepuede.experimental_manager.dict_future_trajectories.get("argentina").dict_sampling_units
d_bol = sisepuede.experimental_manager.dict_future_trajectories.get("bolivia").dict_sampling_units


for k, v in d_bol.items():
    
    vs_arg = v.variable_specifications
    vs_bol = v.variable_specifications

    if vs_arg != vs_bol:
        print(k)

In [91]:
# use the lhs design ONLY from the first region; when running the experiment on the cloud, a single region is run at a time
lhs_design = sisepuede.experimental_manager.dict_lhs_design.get(sorted(regions_run)[0])

for region in regions_run:
    
    
    ftc = sisepuede.experimental_manager.dict_future_trajectories.get(region)
    
    print(f"region: {region}")
    for primary_id in primaries:
        
        # get new strategy id and filter based on primary IDs selected to convert to strategies
        strat_new = dict_primaries_to_strategies.get(primary_id)
        dict_filt_cur = sisepuede.odpt_primary.get_dims_from_key(
            primary_id,
            return_type = "dict",
        )
        
        
        lhs_l, lhs_x, base_future_q = get_lhs_trajes(
            dict_filt_cur,
            region,
            sisepuede,
            lhs_design = lhs_design,
        )
        
        v = lhs_l.get(38)
        print(f"val:\t{v}")
    print("\n")

region: argentina
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: bolivia
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: brazil
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: chile
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: colombia
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: costa_rica
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: dominican_republic
val:	0.8561162382353804
val:	0.8707433110783946
val:	0.6410155649322585
val:	0.9361275537555314
val:	0.7228944436547446


region: ecuador
val:

KeyboardInterrupt: 

In [157]:
importlib.reload(dtr)



if True:
    # set some parameters (WILL SET TO READ FROM A CONFIG OR STRATEGY DEFINITION FILE)
    dict_config_te = {
        "categories_entc_max_investment_ramp": [
            "pp_hydropower",
            "pp_nuclear"
        ],
        "categories_entc_renewable": [
            "pp_geothermal",
            "pp_hydropower",
            "pp_ocean",
            "pp_solar",
            "pp_wind"
        ],
        "categories_inen_high_heat": [
            "cement", 
            "chemicals", 
            "glass", 
            "lime_and_carbonite", 
            "metals"
        ],
        "dict_entc_renewable_target_msp": {
            "pp_solar": 0.15,
            "pp_geothermal": 0.1,
            "pp_wind": 0.15
        },
        "frac_inen_high_temp_elec_hydg": 0.5*0.45,
        "frac_inen_low_temp_elec": 0.95*0.45,
        "n_tp_ramp": None,
        "vir_renewable_cap_delta_frac": 0.01,
        "vir_renewable_cap_max_frac": 0.1,
        "year_0_ramp": year_0_ramp
    }



    transformations_integrated = dtr.TransformationsIntegrated(
        dict_config_te,
        df_input = df_input,
        field_region = field_region,
        id_str = "sisepuede_run_2023-10-08T00:48:31.661121",
        logger = logger,
        regions = regions_run,
    )

2023-10-08 01:07:32,871 - INFO - Successfully initialized Analysis ID 'sisepuede_run_2023-10-08T01:07:32.871127'
2023-10-08 01:07:32,877 - INFO - Verification of SISEPUEDE directory structure completed successfully with 0 errors.
2023-10-08 01:07:34,613 - INFO - NemoMod reference file checks completed successfully.
2023-10-08 01:07:34,614 - INFO - Successfully initialized SISEPUEDEFileStructure.
2023-10-08 01:07:34,618 - INFO - Initializing BaseInputDatabase


MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() to ensure that transition probability rows are enforced on a simplex.

FIX THIS ASAP TO DERIVE PROPERLY.


# Build and export strategies
- include `df_out` from above as `df_exogenous_strategies`; passes adjustments from primary_ids to strategies to transformations_integrated to write output templates

In [160]:
def get_strategies_from_codes(
    strategy_codes: Union[List[str], str],
) -> Union[List[int], None]:
    """
    Map codes to id as input
    """
    attr_strat = sa.model_attributes.get_dimensional_attribute_table(sa.model_attributes.dim_strategy_id)
    dict_map = attr_strat.field_maps.get(f"strategy_code_to_{attr_strat.key}")
    
    # check specification of codes
    strategy_codes = (
        [strategy_codes] 
        if isinstance(strategy_codes, str)
        else (
            strategy_codes
            if sf.islistlike(strategy_codes)
            else None
        )
    )
    
    if strategy_codes is None:
        return None
    
    # get ids to build
    strategies_build = [dict_map.get(x) for x in strategy_codes]
    strategies_build = [x for x in strategies_build if x is not None]
    out = (
        None
        if len(strategies_build) == 0
        else strategies_build
    )
    
    return out



    
strategy_codes_keep = [
    "BASE",
    "LNDU:PLUR",
    "PFLO:BETTER_BASE",
    "PFLO:CHANGE_CONSUMPTION",
    "PFLO:SUPPLY_SIDE_TECH",
    "PFLO:ALL_NO_STOPPING_DEFORESTATION_PLUR",
    "PFLO:ALL_PLUR"
]
strategies_keep = get_strategies_from_codes(strategy_codes_keep)

if True:
    dict_back = transformations_integrated.build_strategies_to_templates(
        df_exogenous_strategies = df_out.rename(columns = {sisepuede.key_region: transformations_integrated.key_region}),
        #regions = ["dominican_republic"],
        #return_q = True,
        strategies = strategies_keep,
    )

   

2023-10-08 01:09:05,134 - INFO - Starting build of 6 strategies in 18 regions...
2023-10-08 01:09:05,179 - INFO - Starting build for region argentina
2023-10-08 01:09:05,193 - INFO - 	Successfully built transformation strategy_id = 1014 ('LNDU: Partial land use reallocation') in 0.01 seconds.
2023-10-08 01:09:08,333 - INFO - 	Successfully built transformation strategy_id = 5001 ('PFLO: Better baseline') in 0.18 seconds.
2023-10-08 01:09:11,569 - INFO - 	Successfully built transformation strategy_id = 5004 ('PFLO: Change consumer behavior') in 0.12 seconds.
2023-10-08 01:09:15,869 - INFO - 	Successfully built transformation strategy_id = 5006 ('PFLO: Supply side technologies and transformations') in 0.62 seconds.
2023-10-08 01:09:20,182 - INFO - 	Successfully built transformation strategy_id = 5007 ('PFLO: All transformations without stopping deforestation and partial land use reallocation') in 0.99 seconds.
2023-10-08 01:09:24,821 - INFO - 	Successfully built transformation strategy_id

In [156]:


for k, v in dict_primaries_to_strategies.items():
    
    dict_filt_cur = sisepuede.odpt_primary.get_dims_from_key(
        k,
        return_type = "dict",
    )
    
    str_print = f"Strategy {v} is converted from ID {sisepuede.id} {sisepuede.key_primary} = {k} ("
    
    for dim, val in dict_filt_cur.items():
        str_print = f"{str_print}{dim}: {val};  "
        
    str_print = f"{str_print.strip()})"
    
    print(str_print)
    

Strategy 5011 is converted from ID sisepuede_run_2023-10-06T20:10:09.932282 primary_id = 514914 (design_id: 3;  strategy_id: 5009;  future_id: 400;)
Strategy 5012 is converted from ID sisepuede_run_2023-10-06T20:10:09.932282 primary_id = 514926 (design_id: 3;  strategy_id: 5009;  future_id: 412;)
Strategy 5013 is converted from ID sisepuede_run_2023-10-06T20:10:09.932282 primary_id = 514978 (design_id: 3;  strategy_id: 5009;  future_id: 464;)
Strategy 5014 is converted from ID sisepuede_run_2023-10-06T20:10:09.932282 primary_id = 515112 (design_id: 3;  strategy_id: 5009;  future_id: 598;)
Strategy 5015 is converted from ID sisepuede_run_2023-10-06T20:10:09.932282 primary_id = 515483 (design_id: 3;  strategy_id: 5009;  future_id: 969;)


In [151]:
sisepuede.id

'sisepuede_run_2023-10-06T20:10:09.932282'